In [1]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense , Flatten , Conv2D , MaxPooling2D , Dropout , BatchNormalization

In [2]:
CIFAR10 = keras.datasets.cifar10

(x_train , y_train) , (x_test , y_test ) = CIFAR10.load_data()
x_train , x_test = x_train/255.0 , x_test/255.0

In [3]:
x_train[2].shape

(32, 32, 3)

# Adding batch size

In [4]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train , y_train)).batch(20).shuffle(buffer_size=1000)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test , y_test)).batch(20)

In [5]:
x_train.shape

(50000, 32, 32, 3)

# Dropout + Batch normalization
increasing accuracy and idea of layer's arrangement taken from :
https://machinelearningmastery.com/how-to-develop-a-cnn-from-scratch-for-cifar-10-photo-classification/

In [13]:
class MyModel(tf.keras.Model):
  def __init__(self):
    super().__init__()
    self.conv2D_1 = Conv2D(64 , (3,3) , activation="relu" , padding="same")
    self.conv2D_2 = Conv2D(64 , (3,3) , activation="relu" , padding="same")
    self.conv2D_3 = Conv2D(128 , (3,3) , activation="relu" , padding="same")
    self.conv2D_4 = Conv2D(128 , (3,3) , activation="relu" , padding="same")
    self.maxpool2D = MaxPooling2D()
    self.dropout = Dropout(0.2)
    self.dropout1 = Dropout(0.3)
    self.dropout2 = Dropout(0.4)
    self.dropout3 = Dropout(0.5)
    self.flatten = Flatten()
    self.dense1 = Dense(128 , activation="relu")
    self.dense = Dense(10 , activation="softmax")
    self.batchnorm1 = BatchNormalization()
    self.batchnorm2 = BatchNormalization()
    self.batchnorm3 = BatchNormalization()
    self.batchnorm4 = BatchNormalization()
    self.batchnorm7 = BatchNormalization()


  def call(self , x):
    x = self.conv2D_1(x)
    x = self.batchnorm1(x)
    x = self.conv2D_2(x)
    x = self.batchnorm2(x)
    x = self.maxpool2D(x)
    x = self.dropout(x)

    x = self.conv2D_3(x)
    x = self.batchnorm3(x)
    x = self.conv2D_4(x)
    x = self.batchnorm4(x)
    x = self.maxpool2D(x)
    x = self.dropout1(x)

    x = self.flatten(x)
    x = self.dense1(x)
    x = self.batchnorm7(x)
    x = self.dropout3(x)
    x = self.dense(x)
    return x

model = MyModel()

In [14]:
# Main loss function
loss_function = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0001)

In [15]:
# Metrics

train_loss = tf.keras.metrics.Mean()
test_loss = tf.keras.metrics.Mean()

train_acc = tf.keras.metrics.SparseCategoricalAccuracy()
test_acc = tf.keras.metrics.SparseCategoricalAccuracy()

# Train

In [16]:
@tf.function
def train(images , labels):
    with tf.GradientTape() as tape :
      prediction = model(images)
      loss = loss_function(y_true=labels , y_pred=prediction)

    gradients = tape.gradient(loss , model.trainable_variables)
    optimizer.apply_gradients(grads_and_vars= zip(gradients , model.trainable_variables))

    train_loss(loss)
    train_acc(labels, prediction)

In [17]:
@tf.function
def test(images , labels):
  prediction = model(images)
  loss = loss_function(y_true=labels , y_pred=prediction)
  test_loss(loss)
  test_acc(labels , prediction)

In [18]:
epochs = 12
for epoch in range(epochs):
  train_loss.reset_states()
  train_acc.reset_states()
  test_loss.reset_states()
  test_acc.reset_states()

  #train
  for images , labels in train_dataset :
    train(images , labels)

  # test
  for images , labels in test_dataset :
    test(images , labels)


  print("epoch:" , epoch + 1  ,
        f"Train Loss: {train_loss.result()}" ,
        f"Train Acc: {train_acc.result()}" ,
        f"Test loss: {test_loss.result()}",
        f"Test Acc: {test_acc.result()}")

epoch: 1 Train Loss: 1.5578081607818604 Train Acc: 0.4380599856376648 Test loss: 1.2910444736480713 Test Acc: 0.5289999842643738
epoch: 2 Train Loss: 1.1145458221435547 Train Acc: 0.6075800061225891 Test loss: 1.0006542205810547 Test Acc: 0.6455000042915344
epoch: 3 Train Loss: 0.8965998888015747 Train Acc: 0.6862599849700928 Test loss: 0.8669480681419373 Test Acc: 0.6965000033378601
epoch: 4 Train Loss: 0.7604389190673828 Train Acc: 0.7348600029945374 Test loss: 0.8365007638931274 Test Acc: 0.7067999839782715
epoch: 5 Train Loss: 0.6516876220703125 Train Acc: 0.7751200199127197 Test loss: 0.752938449382782 Test Acc: 0.7369999885559082
epoch: 6 Train Loss: 0.558782696723938 Train Acc: 0.8057799935340881 Test loss: 0.7206777334213257 Test Acc: 0.7541000247001648
epoch: 7 Train Loss: 0.4710468649864197 Train Acc: 0.8385400176048279 Test loss: 0.6961079239845276 Test Acc: 0.7677000164985657
epoch: 8 Train Loss: 0.392826110124588 Train Acc: 0.8668400049209595 Test loss: 0.7193120121955872 

# best test accuracy :  0.773






+ # decresing batch size from 32 to 20 , increased test accuracy during epochs
+ # decreasing lr in RMSprop , helped a little .
+ # increasing number of filters in conv2D , helped a little .

In [19]:
model.save("cifar10_model")